In [3]:
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ['NEO4J_URI'] = os.getenv("NEO4J_URI")  ###Used for LangSmith tracking
os.environ['NEO4J_USERNAME'] = os.getenv("NEO4J_USERNAME")
os.environ['NEO4J_PASSWORD'] = os.getenv("NEO4J_PASSWORD")

In [5]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph()
graph

In [25]:
##Dataset 

movie_query = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row

MERGE (m:Movie {id: row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)

FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name: trim(director)})
    MERGE (p)-[:DIRECTED]->(m))

FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name: trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))

FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name: trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))



"""

In [33]:
graph.query(movie_query)

[]

In [35]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CEO {name: STRING, YOB: INTEGER}
Company {nmae: STRING}
Student {name: STRING, YOB: INTEGER, POB: STRING}
Country {name: STRING}
User {name: STRING, city: STRING, userId: INTEGER, age: INTEGER}
Post {postId: INTEGER, content: STRING, timestamp: DATE_TIME}
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:User)-[:POSTED]->(:Post)
(:User)-[:LIKES]->(:User)
(:User)-[:FRIEND]->(:User)
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)
